In [1]:
import pandas as pd
import numpy as np
import os
from os import path

In [2]:
symbols = pd.read_csv("D:\Trading\sp500_symbols_list.csv")

In [3]:
tickers_path = "D:\\Trading\\raw_data\\tickers"

In [5]:
tickers_dfs = {ticker: pd.read_csv(path.join(tickers_path, f"{ticker}.csv"), parse_dates=True, index_col="Date") for ticker in symbols['Symbol']}

## Двухдневное направление

In [8]:
merged_closes = pd.DataFrame(index=tickers_dfs[symbols["Symbol"].iloc[0]].index)
for symbol in symbols["Symbol"]:
    merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change()

C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\3677534723.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change()
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\3677534723.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change()
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\3677534723.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [10]:
merged_closes = merged_closes.iloc[1:]

### Бинарное направление

In [63]:
one_dir_growth = np.array([(tickers_dfs[ticker]["Close"].pct_change() > 0).mean() for ticker in symbols['Symbol']])

In [ ]:
pd.Series(one_dir_growth).describe()

count    222.000000
mean       0.515038
std        0.010670
min        0.490128
25%        0.508268
50%        0.514346
75%        0.521542
max        0.552293
dtype: float64

### Значимое изменение

In [14]:
abs_merged_closes = np.abs(merged_closes)

In [24]:
quantiles_dict = {}
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantiles_dict[quantile] = abs_merged_closes.quantile(quantile)
quantiles_df = pd.DataFrame(quantiles_dict)

In [35]:
typical_quantile_list = []
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantile_vals = quantiles_df[quantile]
    perc_10 = quantile_vals.quantile(0.1)
    perc_90 = quantile_vals.quantile(0.9)
    typical_quantile = quantile_vals[(quantile_vals >= perc_10) & (quantile_vals <= perc_90)].mean()
    typical_quantile_list.append(typical_quantile)
    print(f"quantile {quantile}: {typical_quantile}")

quantile 0.1: 0.0017721637266252088
quantile 0.2: 0.0036488995623145688
quantile 0.3: 0.005664746721797614
quantile 0.4: 0.007914917229346714
quantile 0.5: 0.010473913318502141
quantile 0.6: 0.013546531799860161
quantile 0.7: 0.017480580288471607
quantile 0.8: 0.02323024336111691
quantile 0.9: 0.033877524650420834


In [69]:
insign_val = 0.001
dir_dict = {"рост": [],
           "падение": [],
           "незначимо": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change().iloc[1:]
    dir_dict["рост"].append((pct_change >= insign_val).mean())
    dir_dict["падение"].append((pct_change <= -insign_val).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,рост,падение,незначимо
count,222.000000,222.000000,222.000000
mean,0.489933,0.451447,0.058620
std,0.009808,0.013244,0.012796
min,0.461609,0.418792,0.029777
25%,0.483956,0.442213,0.049420
50%,0.488728,0.451539,0.058582
75%,0.495269,0.460713,0.066809
max,0.532797,0.484698,0.094784


### Трехуровневая дифференциация

Незначительное изменение, значимое и сильное

In [74]:
insign_val = 0.001
strong_val = 0.01
dir_dict = {"сильный рост": [],
            "значимый рост": [],
            "незначимо": [],
           "значимое падение": [],
           "сильное падение": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change().iloc[1:]
    dir_dict["сильный рост"].append((pct_change >= strong_val).mean())
    dir_dict["значимый рост"].append(((pct_change < strong_val) & (pct_change >= insign_val)).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
    dir_dict["значимое падение"].append(((pct_change > -strong_val) & (pct_change <= -insign_val)).mean())
    dir_dict["сильное падение"].append((pct_change <= -strong_val).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,сильный рост,значимый рост,незначимо,значимое падение,сильное падение
count,222.000000,222.000000,222.000000,222.000000,222.000000
mean,0.269123,0.220810,0.058620,0.206675,0.244772
std,0.042424,0.039683,0.012796,0.033058,0.042146
min,0.176896,0.100100,0.029777,0.090757,0.147432
25%,0.237823,0.193198,0.049420,0.184960,0.213394
50%,0.268387,0.220433,0.058582,0.209890,0.243623
75%,0.297534,0.248231,0.066809,0.230747,0.270541
max,0.390724,0.314191,0.094784,0.279743,0.384051


## Трёхдневное направление

In [114]:
merged_closes = pd.DataFrame(index=tickers_dfs[symbols["Symbol"].iloc[0]].index)
for symbol in symbols["Symbol"]:
    merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=2)
merged_closes = merged_closes.iloc[2:]

C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\1253389793.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=2)
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\1253389793.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=2)
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\1253389793.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

### Бинарное направление

In [115]:
one_dir_growth = np.array([(tickers_dfs[ticker]["Close"].pct_change(periods=2) > 0).iloc[2:].mean() for ticker in symbols['Symbol']])

In [116]:
pd.Series(one_dir_growth).describe()

count    222.000000
mean       0.526613
std        0.012603
min        0.493324
25%        0.519272
50%        0.525510
75%        0.533903
max        0.564188
dtype: float64

### Значимое изменение

In [117]:
abs_merged_closes = np.abs(merged_closes)

In [118]:
quantiles_dict = {}
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantiles_dict[quantile] = abs_merged_closes.quantile(quantile)
quantiles_df = pd.DataFrame(quantiles_dict)

In [119]:
typical_quantile_list = []
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantile_vals = quantiles_df[quantile]
    perc_10 = quantile_vals.quantile(0.1)
    perc_90 = quantile_vals.quantile(0.9)
    typical_quantile = quantile_vals[(quantile_vals >= perc_10) & (quantile_vals <= perc_90)].mean()
    typical_quantile_list.append(typical_quantile)
    print(f"quantile {quantile}: {typical_quantile}")

quantile 0.1: 0.0026362006283387977
quantile 0.2: 0.005371949596798589
quantile 0.3: 0.008289980561815406
quantile 0.4: 0.011497807001220441
quantile 0.5: 0.015119351898899857
quantile 0.6: 0.01948365072814858
quantile 0.7: 0.02506424005343333
quantile 0.8: 0.032999958057706225
quantile 0.9: 0.04779166493542042


In [120]:
insign_val = 0.01
dir_dict = {"рост": [],
           "падение": [],
           "незначимо": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change(periods=2).iloc[2:]
    dir_dict["рост"].append((pct_change >= insign_val).mean())
    dir_dict["падение"].append((pct_change <= -insign_val).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,рост,падение,незначимо
count,222.000000,222.000000,222.000000
mean,0.342676,0.297177,0.360147
std,0.035799,0.038748,0.072363
min,0.260369,0.205468,0.146195
25%,0.317396,0.269542,0.311726
50%,0.343153,0.295387,0.362492
75%,0.369075,0.320276,0.412287
max,0.433244,0.420561,0.525545


### Трехуровневая дифференциация

Незначительное изменение, значимое и сильное

In [121]:
insign_val = 0.01
strong_val = 0.025
dir_dict = {"сильный рост": [],
            "значимый рост": [],
            "незначимо": [],
           "значимое падение": [],
           "сильное падение": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change(periods=2).iloc[2:]
    dir_dict["сильный рост"].append((pct_change >= strong_val).mean())
    dir_dict["значимый рост"].append(((pct_change < strong_val) & (pct_change >= insign_val)).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
    dir_dict["значимое падение"].append(((pct_change > -strong_val) & (pct_change <= -insign_val)).mean())
    dir_dict["сильное падение"].append((pct_change <= -strong_val).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,сильный рост,значимый рост,незначимо,значимое падение,сильное падение
count,222.000000,222.000000,222.000000,222.000000,222.000000
mean,0.160820,0.181857,0.360147,0.153797,0.143380
std,0.047886,0.017221,0.072363,0.010217,0.044304
min,0.071803,0.099132,0.146195,0.103805,0.065175
25%,0.124136,0.174333,0.311726,0.148206,0.109410
50%,0.153785,0.184332,0.362492,0.155421,0.137673
75%,0.194616,0.193223,0.412287,0.161331,0.167882
max,0.334112,0.213779,0.525545,0.175861,0.316756


## Изменение в 5-дневном интервале

In [122]:
merged_closes = pd.DataFrame(index=tickers_dfs[symbols["Symbol"].iloc[0]].index)
for symbol in symbols["Symbol"]:
    merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=4)
merged_closes = merged_closes.iloc[4:]

C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\4105977521.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=4)
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\4105977521.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_closes[symbol] = tickers_dfs[symbol]["Close"].pct_change(periods=4)
C:\Users\Dell\AppData\Local\Temp\ipykernel_15676\4105977521.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

### Бинарное направление

In [123]:
one_dir_growth = np.array([(tickers_dfs[ticker]["Close"].pct_change(periods=4) > 0).iloc[4:].mean() for ticker in symbols['Symbol']])

In [124]:
pd.Series(one_dir_growth).describe()

count    222.000000
mean       0.539139
std        0.015523
min        0.499012
25%        0.528976
50%        0.539101
75%        0.548197
max        0.578688
dtype: float64

### Значимое изменение

In [125]:
abs_merged_closes = np.abs(merged_closes)

In [126]:
quantiles_dict = {}
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantiles_dict[quantile] = abs_merged_closes.quantile(quantile)
quantiles_df = pd.DataFrame(quantiles_dict)

In [127]:
typical_quantile_list = []
for i in range(1, 10):
    quantile = np.round(0.1 * i, 1)
    quantile_vals = quantiles_df[quantile]
    perc_10 = quantile_vals.quantile(0.1)
    perc_90 = quantile_vals.quantile(0.9)
    typical_quantile = quantile_vals[(quantile_vals >= perc_10) & (quantile_vals <= perc_90)].mean()
    typical_quantile_list.append(typical_quantile)
    print(f"quantile {quantile}: {typical_quantile}")

quantile 0.1: 0.003871995963374894
quantile 0.2: 0.007866771735732879
quantile 0.3: 0.012106867985004367
quantile 0.4: 0.016715209198222385
quantile 0.5: 0.021924988563229303
quantile 0.6: 0.02813837627363973
quantile 0.7: 0.03599501460164953
quantile 0.8: 0.047151141537535635
quantile 0.9: 0.06719053471080834


In [129]:
insign_val = 0.02
dir_dict = {"рост": [],
           "падение": [],
           "незначимо": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change(periods=4).iloc[4:]
    dir_dict["рост"].append((pct_change >= insign_val).mean())
    dir_dict["падение"].append((pct_change <= -insign_val).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,рост,падение,незначимо
count,222.000000,222.000000,222.000000
mean,0.292229,0.238874,0.468896
std,0.045595,0.044674,0.088125
min,0.196411,0.143133,0.201737
25%,0.258685,0.206742,0.407695
50%,0.290583,0.235594,0.470530
75%,0.325445,0.263399,0.535109
max,0.419172,0.379092,0.650456


### Трехуровневая дифференциация

Незначительное изменение, значимое и сильное

In [131]:
insign_val = 0.02
strong_val = 0.04
dir_dict = {"сильный рост": [],
            "значимый рост": [],
            "незначимо": [],
           "значимое падение": [],
           "сильное падение": []}
for ticker in symbols['Symbol']:
    pct_change = tickers_dfs[ticker]["Close"].pct_change(periods=4).iloc[4:]
    dir_dict["сильный рост"].append((pct_change >= strong_val).mean())
    dir_dict["значимый рост"].append(((pct_change < strong_val) & (pct_change >= insign_val)).mean())
    dir_dict["незначимо"].append(((pct_change > -insign_val) & (pct_change < insign_val)).mean())
    dir_dict["значимое падение"].append(((pct_change > -strong_val) & (pct_change <= -insign_val)).mean())
    dir_dict["сильное падение"].append((pct_change <= -strong_val).mean())
dir_df = pd.DataFrame(dir_dict)
dir_df.describe()

,сильный рост,значимый рост,незначимо,значимое падение,сильное падение
count,222.000000,222.000000,222.000000,222.000000,222.000000
mean,0.143133,0.149096,0.468896,0.118098,0.120776
std,0.049678,0.012797,0.088125,0.008031,0.043120
min,0.055540,0.084836,0.201737,0.093186,0.043106
25%,0.105738,0.141928,0.407695,0.112858,0.086768
50%,0.135496,0.150664,0.470530,0.118170,0.114093
75%,0.179010,0.157228,0.535109,0.123689,0.143727
max,0.334335,0.176675,0.650456,0.136154,0.285905
